# Quantifying impact of a News on the volume of the Equity:
It is very obvious that a News about a company positive or negative will affect the company's stock in some manner.
The price can rise-up or fall-down, even stay about the same at times. We can determine the sentiment of the news by the change in price, as to check wether the news was positive or negative.

But to measure the impact a news carry we should use volume as one of the factor. Volume of a stock is very volatile, and thus, it is difficult to analyse the impact by simply calculating percentage change in the volume itself. Believe me when I say that I have observed more than 400% change in volume within some interval of time.

Here is my approach to study the impact news carry by using the volume data before and  after the news. Here I have taken Volume data for 30 mins before and 15 mins after the news. We then comare the mean of volume after the news hits with the mean and standard devitions of before it.

Here, we take news of Infosys(INFY) and volume data for the same. The following code genereates the plots for volume data and categories them into respective directories accordingly. The farther the mean of the data from the earlier mean, the more impactful the news.

The categories I made are :

    1) Moderate ( m-2s, m-s) U  (m+s, m+2s)
    2) Impactful (-infy, m-2s) U (m+2s, infy)

where m : mean (previous 30 mins)
and, s : standard deviation (same)

Files Used :-

    1) Infosys News.csv    
    2) Infosys Stock.csv

In [ ]:
import numpy as np
import pandas as pd
import csv
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import sys
from IPython.display import clear_output

def check(time):
    try:
        s = int(time[11:13]+time[14:16])
        if(s>=945 and s<=1459):
            return True
        else:
            return False
    except:
        print ("except")
        pass

news = open(r'H:\SaberTooth\Analysis\Volume-Weight\Infosys data\INFOSYS News.csv','r',encoding='ISO-8859-1')
readN = csv.reader(news)
next(readN, None)
before, after = 60, 30
dataAvail = pd.to_datetime('2014-03-03T09:15:00+0530')
plotN = 1;

for j in readN :
    time = j[4]
    title = j[6]
    temp = pd.to_datetime(time)
    day = temp.strftime("%A")
    
    time = str(temp)
    time =  time[:10] + 'T' + time[11:] + '+0530'

    if temp < dataAvail :
        continue
    if (not check(time)):
        #print('->>Market closed for input time : ', time)
        continue
    if (day=='Sunday' or day=='Saturday'):
        #print('->>Weekend ')
        continue

    flag=0    
    indB, indA = 0,0
    bef = np.zeros(before)
    aft = np.zeros(after)

    quant = open(r'H:\SaberTooth\Analysis\Volume-Weight\Infosys data\INFOSYS Stock.csv','r',encoding='ISO-8859-1')
    readQ = csv.reader(quant)
    next(readQ, None)
    
    for i in readQ:      
        if flag==0:
            bef[indB] = int(i[6])
            indB = (indB+1)%before
        
        if flag==1:
            aft[indA] = int(i[6])
            indA = (indA+1)%after
            if indA==0:
                break
                
        if time == str(i[1]):
            flag=1
            #print('Match found!')

    quant.close()
    if aft[0] == 0 :
        #print('Holiday')
        continue
    
    print('\nTime : ',time)
    print('News : ', title)
        
    befOrder = np.ones(before)
    for i in np.arange(before):
        befOrder[i] = bef[indB]
        indB = (indB+1)%before

    meanB = np.mean(befOrder)
    stdB = np.std(befOrder)
    meanA = np.mean(aft)

    data = list(befOrder) + list(aft)
    timeList = [ temp + pd.Timedelta(minutes = i) for i in range(-60,30)]
    
    fig = plt.figure(figsize=(15,10))
    plt.plot(timeList,data)
    plt.plot(timeList[:before], np.ones(before) * meanB, color ='black', label = 'mean 1hr before')
    plt.plot(timeList[:before], np.ones(before) * (meanB-stdB), color ='purple', linestyle='--')
    plt.plot(timeList[:before], np.ones(before) * (meanB+stdB), color ='purple', linestyle='--')
    plt.plot(timeList[:before], np.ones(before) * (meanB- 2*stdB), color ='red', linestyle='--')
    plt.plot(timeList[:before], np.ones(before) * (meanB+ 2*stdB), color ='red', linestyle='--')
    plt.plot(timeList[before:], np.ones(after) * meanA, color ='green', label = 'mean 30 min afters')
    plt.xlabel('time')
    plt.ylabel('Volume')
    plt.title(str(time)[:-5] +' := '+ str(title))
    #plt.show()
    
    if(meanA > meanB -0.75*stdB and meanA < meanB +0.75*stdB) :
        continue  #ignore
    elif (meanA > meanB -125*stdB and meanA < meanB +1.25*stdB) :
        #moderately impactful news
        #print('Moderate')
        fig.savefig('H:\SaberTooth\Analysis\Volume-Weight\Output Plots\Infosys\Moderate\plot{0}.png'.format(plotN), dpi=300)   # save the figure to file
        plotN += 1
        plt.close(fig)
    else :
        #highly Impactful
        #print('Garam News')
        fig.savefig('H:\SaberTooth\Analysis\Volume-Weight\Output Plots\Infosys\Impactful\plot{0}.png'.format(plotN), dpi=300)   # save the figure to file
        plotN += 1
        plt.close(fig)
        
    #x = input("Press 1 for next news , 0 to exit")
    #if int(x)==0 :
    #    sys.exit()
    #clear_output()
    
news.close()